In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
#read data
quantile_calib_df = pd.read_csv('data\Financial Timeseries\quantile_calib.csv') 
print(quantile_calib_df.head())
quantile_test_df = pd.read_csv('data\Financial Timeseries\quantile_test.csv')
print(quantile_test_df.head())

pred_calib_df = pd.read_csv('data\Financial Timeseries\pred_calib.csv')
print(pred_calib_df.head())
pred_test_df = pd.read_csv('data\Financial Timeseries\pred_test.csv')
print(pred_test_df.head())

         Date Ticker  pred_lower_calib  pred_upper_calib   y_calib
0  2013-04-01   AAPL         -0.025760          0.026269 -0.031062
1  2013-04-02   AAPL         -0.028656          0.024027  0.002052
2  2013-04-03   AAPL         -0.029541          0.035599  0.005119
3  2013-04-04   AAPL         -0.026721          0.033349 -0.009885
4  2013-04-05   AAPL         -0.024553          0.027757 -0.010567
         Date Ticker  pred_lower_test  pred_upper_test    y_test
0  2020-03-30   AAPL        -0.051964         0.042364  0.028538
1  2020-03-31   AAPL        -0.042124         0.058632 -0.002041
2  2020-04-01   AAPL        -0.042603         0.049162 -0.052617
3  2020-04-02   AAPL        -0.048158         0.053258  0.016687
4  2020-04-03   AAPL        -0.059014         0.053257 -0.014371
         Date Ticker  pred_calib   y_calib
0  2013-04-01   AAPL    0.000785 -0.031062
1  2013-04-02   AAPL    0.003100  0.002052
2  2013-04-03   AAPL   -0.000336  0.005119
3  2013-04-04   AAPL    0.002116 -0.

In [3]:
# coverage rate QR calibration
violation_lower_calib = quantile_calib_df['y_calib'] < quantile_calib_df['pred_lower_calib']
violation_upper_calib = quantile_calib_df['y_calib'] > quantile_calib_df['pred_upper_calib']
violation_calib = violation_lower_calib | violation_upper_calib
coverage_rate_calib = 1 - violation_calib.mean()
print(f'Coverage rate QR calibration: {coverage_rate_calib:.4f}')

# coverage rate QR test
violation_lower_test = quantile_test_df['y_test'] < quantile_test_df['pred_lower_test']
violation_upper_test = quantile_test_df['y_test'] > quantile_test_df['pred_upper_test']
violation_test = violation_lower_test | violation_upper_test
coverage_rate_test = 1 - violation_test.mean()
print(f'Coverage rate QR test: {coverage_rate_test:.4f}')

#give coverage for before conformalisation

Coverage rate QR calibration: 0.8716
Coverage rate QR test: 0.8502


In [4]:
# R^2 and MAE mean calibration
from sklearn.metrics import r2_score, mean_absolute_error
r2_calib = r2_score(pred_calib_df['y_calib'], pred_calib_df['pred_calib'])
mae_calib = mean_absolute_error(pred_calib_df['y_calib'], pred_calib_df['pred_calib'])
print(f'R^2 mean calibration: {r2_calib:.4f}')
print(f'MAE mean calibration: {mae_calib:.4f}')

# R^2 and MAE mean test
r2_test = r2_score(pred_test_df['y_test'], pred_test_df['pred_test'])
mae_test = mean_absolute_error(pred_test_df['y_test'], pred_test_df['pred_test'])
print(f'R^2 mean test: {r2_test:.4f}')
print(f'MAE mean test: {mae_test:.4f}')

R^2 mean calibration: -0.0109
MAE mean calibration: 0.0102
R^2 mean test: -0.0174
MAE mean test: 0.0137


In [5]:
from ConformalPredictor import ConformalPredictor

In [6]:
ticker_list = pred_test_df['Ticker'].unique()
print(f'Tickers: {ticker_list}')
print(f'Number of tickers: {len(ticker_list)}')

#get ticker with the most rows in pred_calib_df
ticker_most_rows = pred_calib_df['Ticker'].value_counts().idxmax()
print(f'Ticker with most rows in pred_calib_df: {ticker_most_rows}')
print(f'Number of rows in pred_calib_df for ticker {ticker_most_rows}: {len(pred_calib_df[pred_calib_df["Ticker"] == ticker_most_rows])}')

# get all tickers with 1762 rows in pred_calib_df
tickers_1762_rows = pred_calib_df['Ticker'].value_counts()[pred_calib_df['Ticker'].value_counts() == 1702].index.tolist()
ticker_list = tickers_1762_rows
print(f'Tickers with 1762 rows in pred_calib_df: {tickers_1762_rows}')
print(f'Number of tickers with 1702 rows in pred_calib_df: {len(tickers_1762_rows)}')
#print num tickers list
print(f'Number of tickers list: {len(ticker_list)}')


Tickers: ['AAPL' 'ABBV' 'ABT' 'ACN' 'ADBE' 'AIG' 'AMD' 'AMGN' 'AMT' 'AMZN' 'AVGO'
 'AXP' 'BA' 'BAC' 'BK' 'BKNG' 'BLK' 'BMY' 'BRK-B' 'C' 'CAT' 'CHTR' 'CL'
 'CMCSA' 'COF' 'COP' 'COST' 'CRM' 'CSCO' 'CVS' 'CVX' 'DE' 'DHR' 'DIS'
 'DUK' 'EMR' 'FDX' 'GD' 'GE' 'GILD' 'GM' 'GOOG' 'GOOGL' 'GS' 'HD' 'HON'
 'IBM' 'INTC' 'INTU' 'ISRG' 'JNJ' 'JPM' 'KO' 'LIN' 'LLY' 'LMT' 'LOW' 'MA'
 'MCD' 'MDLZ' 'MDT' 'MET' 'META' 'MMM' 'MO' 'MRK' 'MS' 'MSFT' 'NEE' 'NFLX'
 'NKE' 'NOW' 'NVDA' 'ORCL' 'PEP' 'PFE' 'PG' 'PLTR' 'PM' 'PYPL' 'QCOM'
 'RTX' 'SBUX' 'SCHW' 'SO' 'SPG' 'T' 'TGT' 'TMO' 'TMUS' 'TSLA' 'TXN' 'UNH'
 'UNP' 'UPS' 'USB' 'V' 'VZ' 'WFC' 'WMT' 'XOM']
Number of tickers: 101
Ticker with most rows in pred_calib_df: AAPL
Number of rows in pred_calib_df for ticker AAPL: 1702
Tickers with 1762 rows in pred_calib_df: ['AAPL', 'PEP', 'NVDA', 'NOW', 'NKE', 'NFLX', 'NEE', 'MSFT', 'MS', 'MRK', 'MO', 'MMM', 'META', 'MET', 'MDT', 'MDLZ', 'MCD', 'MA', 'LOW', 'LMT', 'LLY', 'LIN', 'KO', 'ORCL', 'PFE', 'ISRG', 'PG', 'WMT', '

In [7]:
CongformalPredictor = ConformalPredictor(alpha=0.1)  # Ok (except typo)
ticker_results_standard_CP = []
ticker_results_CQR = []
ticker_results_ACI = []
#split data by ticker
for ticker in ticker_list:
    # mean prediction
    calib_df = pred_calib_df[pred_calib_df['Ticker'] == ticker]
    calib_pred = calib_df['pred_calib']
    y_calib = calib_df['y_calib']
    test_df = pred_test_df[pred_test_df['Ticker'] == ticker]
    pred_test = test_df['pred_test']
    y_test = test_df['y_test']

    # quantile regression prediction
    calib_quantile = quantile_calib_df[quantile_calib_df['Ticker'] == ticker]
    y_calib_quantile = calib_quantile['y_calib']
    pred_lower_calib = calib_quantile['pred_lower_calib']
    pred_upper_calib = calib_quantile['pred_upper_calib']
    test_quantile = quantile_test_df[quantile_test_df['Ticker'] == ticker]
    y_test_quantile = test_quantile['y_test']
    pred_lower_test = test_quantile['pred_lower_test']
    pred_upper_test = test_quantile['pred_upper_test']

    # standard CP
    standard_CP = CongformalPredictor.standard_cp(calib_pred, y_calib, pred_test, y_test)
    ticker_results_standard_CP.append(standard_CP)

    # CQR
    CQR = CongformalPredictor.cqr(pred_lower_calib, pred_upper_calib, y_calib_quantile, pred_lower_test, pred_upper_test, y_test_quantile)
    ticker_results_CQR.append(CQR)

    # ACI
    ACI = CongformalPredictor.aci(calib_pred, y_calib, pred_test, y_test)
    ticker_results_ACI.append(ACI)


In [8]:
# Print Coverage for for each ticker standard CP
print('Coverage for each ticker standard CP:')
for i, ticker in enumerate(ticker_list):
    violations = ticker_results_standard_CP[i]['violation'].sum()
    coverage = 1 - violations / len(ticker_results_standard_CP[i]['violation'])
    print(f'Ticker: {ticker}, Coverage: {coverage:.4f}')
    #get avg. length between lower and upper bound
    avg_length = np.abs(ticker_results_standard_CP[i]['upper_bound'] - ticker_results_standard_CP[i]['lower_bound'])
    avg_length = avg_length.mean()
    print(f'Avg. length: {avg_length:.4f}')

Coverage for each ticker standard CP:
Ticker: AAPL, Coverage: 0.8331
Avg. length: 0.0480
Ticker: PEP, Coverage: 0.8449
Avg. length: 0.0288
Ticker: NVDA, Coverage: 0.7508
Avg. length: 0.0707
Ticker: NOW, Coverage: 0.8671
Avg. length: 0.0734
Ticker: NKE, Coverage: 0.7856
Avg. length: 0.0408
Ticker: NFLX, Coverage: 0.9019
Avg. length: 0.0780
Ticker: NEE, Coverage: 0.7373
Avg. length: 0.0321
Ticker: MSFT, Coverage: 0.8220
Avg. length: 0.0436
Ticker: MS, Coverage: 0.8513
Avg. length: 0.0514
Ticker: MRK, Coverage: 0.8671
Avg. length: 0.0371
Ticker: MO, Coverage: 0.8679
Avg. length: 0.0365
Ticker: MMM, Coverage: 0.8093
Avg. length: 0.0355
Ticker: META, Coverage: 0.8259
Avg. length: 0.0596
Ticker: MET, Coverage: 0.8370
Avg. length: 0.0463
Ticker: MDT, Coverage: 0.8275
Avg. length: 0.0362
Ticker: MDLZ, Coverage: 0.8995
Avg. length: 0.0383
Ticker: MCD, Coverage: 0.8362
Avg. length: 0.0296
Ticker: MA, Coverage: 0.8576
Avg. length: 0.0438
Ticker: LOW, Coverage: 0.8339
Avg. length: 0.0429
Ticker: L

In [9]:
# Print Coverage for for each ticker CQR
print('Coverage for each ticker CQR:')
for i, ticker in enumerate(ticker_list):
    violations = ticker_results_CQR[i]['violation'].sum()
    coverage = 1 - violations / len(ticker_results_CQR[i]['violation'])
    print(f'Ticker: {ticker}, Coverage: {coverage:.4f}')
    #get avg. length between lower and upper bound
    avg_length = np.abs(ticker_results_CQR[i]['upper_bound'] - ticker_results_CQR[i]['lower_bound'])
    avg_length = avg_length.mean()
    print(f'Avg. length: {avg_length:.4f}')

Coverage for each ticker CQR:
Ticker: AAPL, Coverage: 0.8869
Avg. length: 0.0544
Ticker: PEP, Coverage: 0.8853
Avg. length: 0.0333
Ticker: NVDA, Coverage: 0.8513
Avg. length: 0.0885
Ticker: NOW, Coverage: 0.8956
Avg. length: 0.0765
Ticker: NKE, Coverage: 0.8623
Avg. length: 0.0529
Ticker: NFLX, Coverage: 0.9169
Avg. length: 0.0778
Ticker: NEE, Coverage: 0.8718
Avg. length: 0.0478
Ticker: MSFT, Coverage: 0.8790
Avg. length: 0.0507
Ticker: MS, Coverage: 0.8821
Avg. length: 0.0561
Ticker: MRK, Coverage: 0.8916
Avg. length: 0.0405
Ticker: MO, Coverage: 0.8892
Avg. length: 0.0394
Ticker: MMM, Coverage: 0.8734
Avg. length: 0.0443
Ticker: META, Coverage: 0.8639
Avg. length: 0.0673
Ticker: MET, Coverage: 0.8845
Avg. length: 0.0538
Ticker: MDT, Coverage: 0.8758
Avg. length: 0.0428
Ticker: MDLZ, Coverage: 0.8948
Avg. length: 0.0377
Ticker: MCD, Coverage: 0.8845
Avg. length: 0.0338
Ticker: MA, Coverage: 0.8869
Avg. length: 0.0485
Ticker: LOW, Coverage: 0.8892
Avg. length: 0.0496
Ticker: LMT, Cove

In [10]:
# print Coverage for for each ticker ACI
print('Coverage for each ticker ACI:')
for i, ticker in enumerate(ticker_list):
    violations = ticker_results_ACI[i]['violation'].sum()
    coverage = 1 - violations / len(ticker_results_ACI[i]['violation'])
    print(f'Ticker: {ticker}, Coverage: {coverage:.4f}')
    #get avg. length between lower and upper bound
    avg_length = np.abs(ticker_results_ACI[i]['upper_bound'] - ticker_results_ACI[i]['lower_bound'])
    avg_length = avg_length.mean()
    print(f'Avg. length: {avg_length:.4f}')

Coverage for each ticker ACI:
Ticker: AAPL, Coverage: 0.8964
Avg. length: 0.0587
Ticker: PEP, Coverage: 0.8932
Avg. length: 0.0341
Ticker: NVDA, Coverage: 0.8908
Avg. length: 0.1051
Ticker: NOW, Coverage: 0.9043
Avg. length: 0.0829
Ticker: NKE, Coverage: 0.8900
Avg. length: 0.0562
Ticker: NFLX, Coverage: 0.9082
Avg. length: 0.0823
Ticker: NEE, Coverage: 0.8884
Avg. length: 0.0525
Ticker: MSFT, Coverage: 0.8995
Avg. length: 0.0546
Ticker: MS, Coverage: 0.8979
Avg. length: 0.0594
Ticker: MRK, Coverage: 0.8948
Avg. length: 0.0408
Ticker: MO, Coverage: 0.9027
Avg. length: 0.0399
Ticker: MMM, Coverage: 0.8916
Avg. length: 0.0471
Ticker: META, Coverage: 0.8948
Avg. length: 0.0783
Ticker: MET, Coverage: 0.9019
Avg. length: 0.0578
Ticker: MDT, Coverage: 0.8979
Avg. length: 0.0467
Ticker: MDLZ, Coverage: 0.8979
Avg. length: 0.0382
Ticker: MCD, Coverage: 0.8932
Avg. length: 0.0362
Ticker: MA, Coverage: 0.9066
Avg. length: 0.0538
Ticker: LOW, Coverage: 0.8979
Avg. length: 0.0530
Ticker: LMT, Cove

In [11]:
from TestingFramework import binomial_test, geometric_conformal_test, dynamic_binary_regression_test, diebold_mariano_test

In [12]:
#1. Test on standard CP
# Testing
p_values_binom_too_many = []
p_values_binom_too_few = []
p_values_binom_two_sided = []
p_values_geom_UC = []
p_values_geom_ind = []
p_values_geom_joint = []
p_values_dynamic_binary_ind = []
p_values_dynamic_binary_cc = []

for i, ticker in enumerate(ticker_list):
    p_under_binom, p_over_binom, p_two_sid_binom = binomial_test(ticker_results_standard_CP[i]['violation'], alpha=0.1)
    p_values_binom_too_many.append(p_under_binom)
    p_values_binom_too_few.append(p_over_binom)
    p_values_binom_two_sided.append(p_two_sid_binom)
    p_UC, p_ind, p_joint = geometric_conformal_test(ticker_results_standard_CP[i]['violation'], alpha=0.1)
    p_values_geom_UC.append(p_UC)
    p_values_geom_ind.append(p_ind)
    p_values_geom_joint.append(p_joint)
    p_dynamic_binary_ind, p_dynamic_binary_cc = dynamic_binary_regression_test(ticker_results_standard_CP[i]['y_true'], ticker_results_standard_CP[i]['violation'], ticker_results_standard_CP[i]['lower_bound'], ticker_results_standard_CP[i]['upper_bound'], alpha=0.1)
    p_values_dynamic_binary_ind.append(p_dynamic_binary_ind)
    p_values_dynamic_binary_cc.append(p_dynamic_binary_cc)
    


# Print p-values for standard CP
print('P-values for standard CP:')
for i, ticker in enumerate(ticker_list):
    print(f'Ticker: {ticker}')
    print(f'Binomial test too many: {p_values_binom_too_many[i]:.4f}')
    print(f'Binomial test too few: {p_values_binom_too_few[i]:.4f}')
    print(f'Binomial test two sided: {p_values_binom_two_sided[i]:.4f}')
    print(f'Geometric UC test: {p_values_geom_UC[i]:.4f}')
    print(f'Geometric independent test: {p_values_geom_ind[i]:.4f}')
    print(f'Geometric joint test: {p_values_geom_joint[i]:.4f}')
    print(f'Dynamic binary regression test: {p_values_dynamic_binary_ind[i]:.4f}')
    print(f'Dynamic binary regression test CC: {p_values_dynamic_binary_cc[i]:.4f}')
    print('---')





P-values for standard CP:
Ticker: AAPL
Binomial test too many: 0.0000
Binomial test too few: 1.0000
Binomial test two sided: 0.0000
Geometric UC test: 0.0000
Geometric independent test: 0.0000
Geometric joint test: 0.0000
Dynamic binary regression test: 0.0000
Dynamic binary regression test CC: 0.0000
---
Ticker: PEP
Binomial test too many: 0.0000
Binomial test too few: 1.0000
Binomial test two sided: 0.0000
Geometric UC test: 0.0000
Geometric independent test: 0.0070
Geometric joint test: 0.0000
Dynamic binary regression test: 0.0202
Dynamic binary regression test CC: 0.0000
---
Ticker: NVDA
Binomial test too many: 0.0000
Binomial test too few: 1.0000
Binomial test two sided: 0.0000
Geometric UC test: 0.0000
Geometric independent test: 0.0012
Geometric joint test: 0.0000
Dynamic binary regression test: 0.0000
Dynamic binary regression test CC: 0.0000
---
Ticker: NOW
Binomial test too many: 0.0001
Binomial test too few: 0.9999
Binomial test two sided: 0.0002
Geometric UC test: 0.0003
G

In [13]:
#count for each tests the number of p-values > 0.05 (Tests passed)
count_binom_too_many = sum(p > 0.05 for p in p_values_binom_too_many)
count_binom_too_few = sum(p > 0.05 for p in p_values_binom_too_few)
count_binom_two_sided = sum(p > 0.05 for p in p_values_binom_two_sided)
count_geom_UC = sum(p > 0.05 for p in p_values_geom_UC)
count_geom_ind = sum(p > 0.05 for p in p_values_geom_ind)
count_geom_joint = sum(p > 0.05 for p in p_values_geom_joint)
count_dynamic_binary_ind = sum(p > 0.05 for p in p_values_dynamic_binary_ind)
count_dynamic_binary_cc = sum(p > 0.05 for p in p_values_dynamic_binary_cc)
print(f'Number of p-values > 0.05 for binomial test too many Split CP: {count_binom_too_many}')
print(f'Number of p-values > 0.05 for binomial test too few Split CP: {count_binom_too_few}')
print(f'Number of p-values > 0.05 for binomial test two sided Split CP: {count_binom_two_sided}')
print(f'Number of p-values > 0.05 for geometric UC test Split CP: {count_geom_UC}')
print(f'Number of p-values > 0.05 for geometric independent test Split CP: {count_geom_ind}')
print(f'Number of p-values > 0.05 for geometric joint test Split CP: {count_geom_joint}')
print(f'Number of p-values > 0.05 for dynamic binary regression ind test Split CP: {count_dynamic_binary_ind}')
print(f'Number of p-values > 0.05 for dynamic binary regression cc test Split CP: {count_dynamic_binary_cc}')

Number of p-values > 0.05 for binomial test too many Split CP: 8
Number of p-values > 0.05 for binomial test too few Split CP: 94
Number of p-values > 0.05 for binomial test two sided Split CP: 4
Number of p-values > 0.05 for geometric UC test Split CP: 5
Number of p-values > 0.05 for geometric independent test Split CP: 1
Number of p-values > 0.05 for geometric joint test Split CP: 1
Number of p-values > 0.05 for dynamic binary regression ind test Split CP: 8
Number of p-values > 0.05 for dynamic binary regression cc test Split CP: 4


In [14]:
# 2. Test on CQR
# Testing
p_values_binom_too_many_CQR = []
p_values_binom_too_few_CQR = []
p_values_binom_two_sided_CQR = []
p_values_geom_UC_CQR = []
p_values_geom_ind_CQR = []
p_values_geom_joint_CQR = []
p_values_dynamic_binary_ind_CQR = []
p_values_dynamic_binary_cc_CQR = []


for i, ticker in enumerate(ticker_list):
    p_under_binom_CQR, p_over_binom_CQR, p_two_sid_binom_CQR = binomial_test(ticker_results_CQR[i]['violation'], alpha=0.1)
    p_values_binom_too_many_CQR.append(p_under_binom_CQR)
    p_values_binom_too_few_CQR.append(p_over_binom_CQR)
    p_values_binom_two_sided_CQR.append(p_two_sid_binom_CQR)
    p_UC_CQR, p_ind_CQR, p_joint_CQR = geometric_conformal_test(ticker_results_CQR[i]['violation'], alpha=0.1)
    p_values_geom_UC_CQR.append(p_UC_CQR)
    p_values_geom_ind_CQR.append(p_ind_CQR)
    p_values_geom_joint_CQR.append(p_joint_CQR)
    p_dynamic_binary_ind_CQR, p_dynamic_binary_cc_CQR = dynamic_binary_regression_test(ticker_results_CQR[i]['y_true'], ticker_results_CQR[i]['violation'], ticker_results_CQR[i]['lower_bound'], ticker_results_CQR[i]['upper_bound'], alpha=0.1)
    p_values_dynamic_binary_ind_CQR.append(p_dynamic_binary_ind_CQR)
    p_values_dynamic_binary_cc_CQR.append(p_dynamic_binary_cc_CQR)

# Print p-values for CQR
print('P-values for CQR:')  
for i, ticker in enumerate(ticker_list):
    print(f'Ticker: {ticker}')
    print(f'Binomial test too many: {p_values_binom_too_many_CQR[i]:.4f}')
    print(f'Binomial test too few: {p_values_binom_too_few_CQR[i]:.4f}')
    print(f'Binomial test two sided: {p_values_binom_two_sided_CQR[i]:.4f}')
    print(f'Geometric UC test: {p_values_geom_UC_CQR[i]:.4f}')
    print(f'Geometric independent test: {p_values_geom_ind_CQR[i]:.4f}')
    print(f'Geometric joint test: {p_values_geom_joint_CQR[i]:.4f}')
    print(f'Dynamic binary regression ind test: {p_values_dynamic_binary_ind_CQR[i]:.4f}')
    print(f'Dynamic binary regression cc test: {p_values_dynamic_binary_cc_CQR[i]:.4f}')
    print('---')

P-values for CQR:
Ticker: AAPL
Binomial test too many: 0.0675
Binomial test too few: 0.9435
Binomial test two sided: 0.1218
Geometric UC test: 0.1859
Geometric independent test: 0.5346
Geometric joint test: 0.3438
Dynamic binary regression ind test: 0.5442
Dynamic binary regression cc test: 0.4911
---
Ticker: PEP
Binomial test too many: 0.0469
Binomial test too few: 0.9612
Binomial test two sided: 0.0828
Geometric UC test: 0.1324
Geometric independent test: 0.2718
Geometric joint test: 0.1763
Dynamic binary regression ind test: 0.0276
Dynamic binary regression cc test: 0.0205
---
Ticker: NVDA
Binomial test too many: 0.0000
Binomial test too few: 1.0000
Binomial test two sided: 0.0000
Geometric UC test: 0.0000
Geometric independent test: 0.1835
Geometric joint test: 0.0000
Dynamic binary regression ind test: 0.1268
Dynamic binary regression cc test: 0.0002
---
Ticker: NOW
Binomial test too many: 0.3129
Binomial test too few: 0.7191
Binomial test two sided: 0.6059
Geometric UC test: 0.76

In [15]:
#count for each tests the number of p-values > 0.05 (Tests passed) for CQR	
count_binom_too_many_CQR = sum(p > 0.05 for p in p_values_binom_too_many_CQR)
count_binom_too_few_CQR = sum(p > 0.05 for p in p_values_binom_too_few_CQR)
count_binom_two_sided_CQR = sum(p > 0.05 for p in p_values_binom_two_sided_CQR)
count_geom_UC_CQR = sum(p > 0.05 for p in p_values_geom_UC_CQR)
count_geom_ind_CQR = sum(p > 0.05 for p in p_values_geom_ind_CQR)
count_geom_joint_CQR = sum(p > 0.05 for p in p_values_geom_joint_CQR)
count_dynamic_binary_ind_CQR = sum(p > 0.05 for p in p_values_dynamic_binary_ind_CQR)
count_dynamic_binary_cc_CQR = sum(p > 0.05 for p in p_values_dynamic_binary_cc_CQR)

print(f'Number of p-values > 0.05 for binomial test too many CQR: {count_binom_too_many_CQR}')
print(f'Number of p-values > 0.05 for binomial test too few CQR: {count_binom_too_few_CQR}')
print(f'Number of p-values > 0.05 for binomial test two sided CQR: {count_binom_two_sided_CQR}')
print(f'Number of p-values > 0.05 for geometric UC test CQR: {count_geom_UC_CQR}')
print(f'Number of p-values > 0.05 for geometric independent test CQR: {count_geom_ind_CQR}')
print(f'Number of p-values > 0.05 for geometric joint test CQR: {count_geom_joint_CQR}')
print(f'Number of p-values > 0.05 for dynamic binary regression ind test CQR: {count_dynamic_binary_ind_CQR}')
print(f'Number of p-values > 0.05 for dynamic binary regression cc test CQR: {count_dynamic_binary_cc_CQR}')

Number of p-values > 0.05 for binomial test too many CQR: 31
Number of p-values > 0.05 for binomial test too few CQR: 97
Number of p-values > 0.05 for binomial test two sided CQR: 39
Number of p-values > 0.05 for geometric UC test CQR: 41
Number of p-values > 0.05 for geometric independent test CQR: 90
Number of p-values > 0.05 for geometric joint test CQR: 45
Number of p-values > 0.05 for dynamic binary regression ind test CQR: 87
Number of p-values > 0.05 for dynamic binary regression cc test CQR: 63


In [16]:
# 3. Test on ACI
# Testing
p_values_binom_too_many_ACI = []
p_values_binom_too_few_ACI = []
p_values_binom_two_sided_ACI = []
p_values_geom_UC_ACI = []
p_values_geom_ind_ACI = []
p_values_geom_joint_ACI = []
p_values_dynamic_binary_ind_ACI = []
p_values_dynamic_binary_cc_ACI = []

for i, ticker in enumerate(ticker_list):
    p_under_binom_ACI, p_over_binom_ACI, p_two_sid_binom_ACI = binomial_test(ticker_results_ACI[i]['violation'], alpha=0.1)
    p_values_binom_too_many_ACI.append(p_under_binom_ACI)
    p_values_binom_too_few_ACI.append(p_over_binom_ACI)
    p_values_binom_two_sided_ACI.append(p_two_sid_binom_ACI)
    p_UC_ACI, p_ind_ACI, p_joint_ACI = geometric_conformal_test(ticker_results_ACI[i]['violation'], alpha=0.1)
    p_values_geom_UC_ACI.append(p_UC_ACI)
    p_values_geom_ind_ACI.append(p_ind_ACI)
    p_values_geom_joint_ACI.append(p_joint_ACI)
    p_dynamic_binary_ind_ACI, p_dynamic_binary_cc_ACI = dynamic_binary_regression_test(ticker_results_ACI[i]['y_true'], ticker_results_ACI[i]['violation'], ticker_results_ACI[i]['lower_bound'], ticker_results_ACI[i]['upper_bound'], alpha=0.1)
    p_values_dynamic_binary_ind_ACI.append(p_dynamic_binary_ind_ACI)
    p_values_dynamic_binary_cc_ACI.append(p_dynamic_binary_cc_ACI)
    

# Print p-values for ACI
print('P-values for ACI:')
for i, ticker in enumerate(ticker_list):
    print(f'Ticker: {ticker}')
    print(f'Binomial test too many: {p_values_binom_too_many_ACI[i]:.4f}')
    print(f'Binomial test too few: {p_values_binom_too_few_ACI[i]:.4f}')
    print(f'Binomial test two sided: {p_values_binom_two_sided_ACI[i]:.4f}')
    print(f'Geometric UC test: {p_values_geom_UC_ACI[i]:.4f}')
    print(f'Geometric independent test: {p_values_geom_ind_ACI[i]:.4f}')
    print(f'Geometric joint test: {p_values_geom_joint_ACI[i]:.4f}')
    print(f'Dynamic binary regression ind test: {p_values_dynamic_binary_ind_ACI[i]:.4f}')
    print(f'Dynamic binary regression cc test: {p_values_dynamic_binary_cc_ACI[i]:.4f}')
    print('---')

P-values for ACI:
Ticker: AAPL
Binomial test too many: 0.3464
Binomial test too few: 0.6871
Binomial test two sided: 0.6730
Geometric UC test: 0.8440
Geometric independent test: 0.0000
Geometric joint test: 0.0000
Dynamic binary regression ind test: 0.0000
Dynamic binary regression cc test: 0.0001
---
Ticker: PEP
Binomial test too many: 0.2222
Binomial test too few: 0.8042
Binomial test two sided: 0.4253
Geometric UC test: 0.5565
Geometric independent test: 0.0590
Geometric joint test: 0.1415
Dynamic binary regression ind test: 0.0108
Dynamic binary regression cc test: 0.0138
---
Ticker: NVDA
Binomial test too many: 0.1492
Binomial test too few: 0.8710
Binomial test two sided: 0.2807
Geometric UC test: 0.3870
Geometric independent test: 0.0005
Geometric joint test: 0.0015
Dynamic binary regression ind test: 0.0006
Dynamic binary regression cc test: 0.0008
---
Ticker: NOW
Binomial test too many: 0.7069
Binomial test too few: 0.3265
Binomial test two sided: 0.6394
Geometric UC test: 0.50

In [17]:
#count for each tests the number of p-values > 0.05 (Tests passed) for ACI
count_binom_too_many_ACI = sum(p > 0.05 for p in p_values_binom_too_many_ACI)
count_binom_too_few_ACI = sum(p > 0.05 for p in p_values_binom_too_few_ACI)
count_binom_two_sided_ACI = sum(p > 0.05 for p in p_values_binom_two_sided_ACI)
count_geom_UC_ACI = sum(p > 0.05 for p in p_values_geom_UC_ACI)
count_geom_ind_ACI = sum(p > 0.05 for p in p_values_geom_ind_ACI)
count_geom_joint_ACI = sum(p > 0.05 for p in p_values_geom_joint_ACI)
count_dynamic_binary_ind_ACI = sum(p > 0.05 for p in p_values_dynamic_binary_ind_ACI)
count_dynamic_binary_cc_ACI = sum(p > 0.05 for p in p_values_dynamic_binary_cc_ACI)
print(f'Number of p-values > 0.05 for binomial test too many ACI: {count_binom_too_many_ACI}')
print(f'Number of p-values > 0.05 for binomial test too few ACI: {count_binom_too_few_ACI}')
print(f'Number of p-values > 0.05 for binomial test two sided ACI: {count_binom_two_sided_ACI}')
print(f'Number of p-values > 0.05 for geometric UC test ACI: {count_geom_UC_ACI}')
print(f'Number of p-values > 0.05 for geometric independent test ACI: {count_geom_ind_ACI}')
print(f'Number of p-values > 0.05 for geometric joint test ACI: {count_geom_joint_ACI}')
print(f'Number of p-values > 0.05 for dynamic binary regression ind test ACI: {count_dynamic_binary_ind_ACI}')
print(f'Number of p-values > 0.05 for dynamic binary regression cc test ACI: {count_dynamic_binary_cc_ACI}')

Number of p-values > 0.05 for binomial test too many ACI: 96
Number of p-values > 0.05 for binomial test too few ACI: 98
Number of p-values > 0.05 for binomial test two sided ACI: 97
Number of p-values > 0.05 for geometric UC test ACI: 97
Number of p-values > 0.05 for geometric independent test ACI: 10
Number of p-values > 0.05 for geometric joint test ACI: 22
Number of p-values > 0.05 for dynamic binary regression ind test ACI: 36
Number of p-values > 0.05 for dynamic binary regression cc test ACI: 36


In [18]:
# 5. Diebold Marino Test between standard CP, CQR, ACI and adaptive_cqr

# Testing
p_values_diebold_mariano_std_CQR = []
DM_stat_std_CQR = []
p_values_diebold_mariano_std_ACI = []
DM_stat_std_ACI = []
p_values_diebold_mariano_CQR_ACI = []
DM_stat_CQR_ACI = []

for i, ticker in enumerate(ticker_list):
    DM_stat, p_value = diebold_mariano_test(ticker_results_standard_CP[i]['lower_bound'], ticker_results_standard_CP[i]['upper_bound'], ticker_results_CQR[i]['lower_bound'], ticker_results_CQR[i]['upper_bound'], ticker_results_CQR[i]['y_true'], alpha=0.1)
    p_values_diebold_mariano_std_CQR.append(p_value)
    DM_stat_std_CQR.append(DM_stat)
    DM_stat, p_value = diebold_mariano_test(ticker_results_standard_CP[i]['lower_bound'], ticker_results_standard_CP[i]['upper_bound'], ticker_results_ACI[i]['lower_bound'], ticker_results_ACI[i]['upper_bound'], ticker_results_ACI[i]['y_true'], alpha=0.1)
    p_values_diebold_mariano_std_ACI.append(p_value)
    DM_stat_std_ACI.append(DM_stat)
    DM_stat, p_value = diebold_mariano_test(ticker_results_CQR[i]['lower_bound'], ticker_results_CQR[i]['upper_bound'], ticker_results_ACI[i]['lower_bound'], ticker_results_ACI[i]['upper_bound'], ticker_results_ACI[i]['y_true'], alpha=0.1)
    p_values_diebold_mariano_CQR_ACI.append(p_value)
    DM_stat_CQR_ACI.append(DM_stat)


# Print DM stat and p-values for standard CP and CQR
print('DM stat and p-values for standard CP and CQR:')
for i, ticker in enumerate(ticker_list):
    print(f'Ticker: {ticker}')
    print(f'DM stat: {DM_stat_std_CQR[i]:.4f}')
    print(f'p-value: {p_values_diebold_mariano_std_CQR[i]:.4f}')
    print('---')
print('#######################################################################################')
# Print DM stat and p-values for standard CP and ACI
print('DM stat and p-values for standard CP and ACI:')
for i, ticker in enumerate(ticker_list):
    print(f'Ticker: {ticker}')
    print(f'DM stat: {DM_stat_std_ACI[i]:.4f}')
    print(f'p-value: {p_values_diebold_mariano_std_ACI[i]:.4f}')
    print('---')
print('#######################################################################################')
# Print DM stat and p-values for CQR and ACI
print('DM stat and p-values for CQR and ACI:')
for i, ticker in enumerate(ticker_list):
    print(f'Ticker: {ticker}')
    print(f'DM stat: {DM_stat_CQR_ACI[i]:.4f}')
    print(f'p-value: {p_values_diebold_mariano_CQR_ACI[i]:.4f}')
    print('---')

Mean loss differential d̄   : -0.006318
HAC variance (Newey-West)  : 0.000002
Diebold-Mariano statistic  : -4.894
p-value                    : 0.0000
Mean loss differential d̄   : -0.010618
HAC variance (Newey-West)  : 0.000001
Diebold-Mariano statistic  : -9.564
p-value                    : 0.0000
Mean loss differential d̄   : -0.004300
HAC variance (Newey-West)  : 0.000001
Diebold-Mariano statistic  : -3.682
p-value                    : 0.0002
Mean loss differential d̄   : -0.004526
HAC variance (Newey-West)  : 0.000001
Diebold-Mariano statistic  : -5.239
p-value                    : 0.0000
Mean loss differential d̄   : -0.005311
HAC variance (Newey-West)  : 0.000000
Diebold-Mariano statistic  : -18.439
p-value                    : 0.0000
Mean loss differential d̄   : -0.000786
HAC variance (Newey-West)  : 0.000001
Diebold-Mariano statistic  : -0.966
p-value                    : 0.3338
Mean loss differential d̄   : -0.017737
HAC variance (Newey-West)  : 0.000003
Diebold-Mariano stati

In [19]:
# for each pair make 3 cat: method 1 better with p-value > 0.05, method 2 better with p-value, tie (uncertain) 
count_std_CQR_better = 0
count_std_CQR_worse = 0
count_std_CQR_tie = 0
count_std_ACI_better = 0
count_std_ACI_worse = 0
count_std_ACI_tie = 0
count_CQR_ACI_better = 0
count_CQR_ACI_worse = 0
count_CQR_ACI_tie = 0

for i, ticker in enumerate(ticker_list):
    if p_values_diebold_mariano_std_CQR[i] < 0.05:
        if DM_stat_std_CQR[i] < 0:
            count_std_CQR_better += 1
        elif DM_stat_std_CQR[i] > 0:
            count_std_CQR_worse += 1
    else:
        count_std_CQR_tie += 1
        print(f'Ticker: {ticker}')
        print(f'DM stat: {DM_stat_std_CQR[i]:.4f}')
        print(f'p-value: {p_values_diebold_mariano_std_CQR[i]:.4f}')
        print('---')
    if p_values_diebold_mariano_std_ACI[i] < 0.05:
        if DM_stat_std_ACI[i] < 0:
            count_std_ACI_better += 1
        elif DM_stat_std_ACI[i] > 0:
            count_std_ACI_worse += 1
    else:
        count_std_ACI_tie += 1
    if p_values_diebold_mariano_CQR_ACI[i] < 0.05:
        if DM_stat_CQR_ACI[i] < 0:
            count_CQR_ACI_better += 1
        elif DM_stat_CQR_ACI[i] > 0:
            count_CQR_ACI_worse += 1
    else:
        count_CQR_ACI_tie += 1

Ticker: NOW
DM stat: -1.9092
p-value: 0.0562
---
Ticker: NFLX
DM stat: 0.1091
p-value: 0.9131
---
Ticker: MDLZ
DM stat: 0.7504
p-value: 0.4530
---
Ticker: UNP
DM stat: 1.2009
p-value: 0.2298
---
Ticker: AMGN
DM stat: 1.7995
p-value: 0.0719
---
Ticker: AMD
DM stat: 1.0034
p-value: 0.3157
---
Ticker: CSCO
DM stat: -1.0285
p-value: 0.3037
---


In [20]:
print(f'Count standard CP better than CQR: {count_std_CQR_better}')
print(f'Count standard CP worse than CQR: {count_std_CQR_worse}')
print(f'Count standard CP tie with CQR: {count_std_CQR_tie}')
print('#######################################################################################')
print(f'Count standard CP better than ACI: {count_std_ACI_better}')
print(f'Count standard CP worse than ACI: {count_std_ACI_worse}')
print(f'Count standard CP tie with ACI: {count_std_ACI_tie}')
print('#######################################################################################')
print(f'Count CQR better than ACI: {count_CQR_ACI_better}')
print(f'Count CQR worse than ACI: {count_CQR_ACI_worse}')
print(f'Count CQR tie with ACI: {count_CQR_ACI_tie}')


Count standard CP better than CQR: 88
Count standard CP worse than CQR: 3
Count standard CP tie with CQR: 7
#######################################################################################
Count standard CP better than ACI: 91
Count standard CP worse than ACI: 5
Count standard CP tie with ACI: 2
#######################################################################################
Count CQR better than ACI: 70
Count CQR worse than ACI: 1
Count CQR tie with ACI: 27
